In [ ]:
library(devtools)
library(tidyverse)
library(magrittr)
library(readxl)
library(raster)
library(maptools)
library(rgdal)
library(ggplot2)
library(rgeos)
library(foreach)
library(sf)
library(quantreg)

In [ ]:
options("rgdal_show_exportToProj4_warnings"="none")

In [ ]:
install_github('swinersha/UAVforestR')

In [ ]:
library(UAVforestR)

In [ ]:
alm<-read_xlsx("/home/jovyan/lustre_scratch/allometries/GlobalAllometricDatabase.xlsx", sheet ="Data")

alm %<>%
  filter(Biogeographic_zone == "Indo-Malaya",
         Biome == "Tropical forests")

lut<-rq_lut(x=alm$H, y=alm$CD/2, log=TRUE)

lut[50,]

In [ ]:
crown_overlap<-function(auto_trees, manual_trees, buffer_by, verbose='off'){
  # out<-matrix(0, nrow=nrow(manual_trees), ncol=4)
  out<-matrix(0, nrow=nrow(manual_trees), ncol=6)
  out[,1]<-1:nrow(manual_trees)
  out[,3]<-1
  
  auto_trees$id<-1:nrow(auto_trees)
  sum(sapply(auto_trees@polygons, function(x) x@area)<0.001)
  for(i in 1:nrow(manual_trees)){
    i_esc<<-i
    #    print(i)
    poly<-manual_trees[i,] # selects the current polygon
    poly_area<-poly@polygons[[1]]@Polygons[[1]]@area # the area of the polygon
    poly_buffer<-gBuffer(poly, width=buffer_by) # makes a buffer around it
    #cropped_trees<-raster::crop(auto_trees, poly_buffer) # crops the auto trees to the buffer
    cropped_trees<-raster::crop(auto_trees, poly) # crops the auto trees to any that intersect with poly
    cropped_trees<-auto_trees[auto_trees$id %in% cropped_trees$id,]
    cropped_trees <- gBuffer(cropped_trees, byid=TRUE, width=0) # deals with self-intersection
    if(!is.null(cropped_trees)){
      for (j in 1:nrow(cropped_trees)) {
        #      print(j)
        overlap <- gIntersection(poly, cropped_trees[j, ])# extracts the intersecting area.
        overlap_area <- overlap@polygons[[1]]@Polygons[[1]]@area # the area of the intersection
        union1 <- gUnion(poly, cropped_trees[j, ])
        union1_area <- union1@polygons[[1]]@Polygons[[1]]@area
        iou <- overlap_area/union1_area
        if (iou > 0.49) {
          auto_area <- cropped_trees@polygons[[j]]@Polygons[[1]]@area
          overlap_area <- overlap@polygons[[1]]@Polygons[[1]]@area # the area of the intersection
          
          #overseg <- (auto_area - overlap_area) / poly_area
          overseg <- 1-(overlap_area / auto_area) # false positive rate
          underseg <- (poly_area - overlap_area) / poly_area # false negative rate
          # overlap_percent<-overlap_area/poly_area # the percentage area
          size_ratio<-auto_area/poly_area # the percentage area
          
          if (out[i, 3] == 1) {
            out[i, 2] <- cropped_trees$id[j]
            out[i, 3] <- 0
            out[i, 4] <- overseg
            out[i, 5] <- underseg
            out[i, 6] <- size_ratio
          }
          else if ((overseg + underseg) < sum(out[i, 4:5])) {
            # stores the result
            out[i, 2] <- cropped_trees$id[j]
            out[i, 4] <- overseg
            out[i, 5] <- underseg
            out[i, 6] <- size_ratio
          }
          # if(verbose=='on')
          #   cat('j: ', j, 'overlap: ', overlap_percent, '\n')
          # if(overlap_percent>out[i,3]){ # stores the result
          #   out[i,1]<-i
          #   out[i,2]<-cropped_trees$id[j]
          #   out[i,3]<-overlap_percent
          #   out[i,4]<-size_ratio
          # }
        }
      }
      if (verbose == 'on')
        cat('out: ', out[i, ], '\n')
    }
  }
  out[out[,2]==0,2]<-NA
  # Loads these as additional columns for the manual trees:
  manual_trees$id_auto_tree<-out[,2]
  # manual_trees$overlap_auto_tree <- out[, 3]
  # manual_trees$size_ratio <- out[, 4]
  manual_trees$tree_match <- out[, 3]
  manual_trees$overseg <- out[, 4]
  manual_trees$underseg <- out[, 5]
  manual_trees$cost <- rowSums(out[,4:5])/2 + out[,3]
  manual_trees$size_ratio<-out[,6]
  
  if (verbose == 'on')
    cat('Cost: ', manual_trees$cost, '\n')
  
  return(manual_trees)
}


In [ ]:
### One cell version

### Now want to run these automatically, where these new values are read in from another script.
### In fact, we could just amalgamate into a single script. However they are in two different languages!
### So will need to run them separately.
new_bo_params = read.csv('/home/jovyan/bayesian_opt/new_bo_params.csv', check.names=FALSE, header=FALSE)


THRESHSeed_vec<-0.6
THRESHCrown_vec<-new_bo_params[1,1]
SOBELstr_vec<-new_bo_params[1,2]
tau_vec<-new_bo_params[1,3]
#searchwin_vec<-seq(from=3, to=9, by=6)
params<-expand.grid(THRESHSeed=THRESHSeed_vec,
                    THRESHCrown=THRESHCrown_vec,
                    SOBELstr=SOBELstr_vec,
                    tau=tau_vec) #,
# searchwin=searchwin_vec)

# Toms UAVforestR ############

# Set up look up table
#alm<-read_xlsx("C:/Users/sebhi/ai4er/mres_project/work/data/GlobalAllometricDatabase.xlsx", sheet ="Data")
#alm %<>%  filter(Biogeographic_zone == "Indo-Malaya", Biome == "Tropical forests")
#lut<-rq_lut(x=alm$H, y=alm$CD/2, log=TRUE)

lid_chm<-raster("/home/jovyan/lustre_scratch/sepilok_data/sepilok_2014_lidar/Sep_2014_coarse_CHM_g10_sub0.01_0.5m.tif")
lid_chm = crop(lid_chm, extent(603450, 603650, 648000, 648200))

# Set up imagery
chm_blur<-blur(lid_chm)
#plot(chm_blur)
chm_sobel<-sobel_edge(lid_chm)
#plot(chm_sobel)
imagery=chm_blur; im_sobel=chm_sobel;  pypath = NULL

# Ok, this works thus far. Now let's try to include the search params.

#out<-vector(mode="list", length=nrow(params))

# Separate treetops and segmentation
tau = 40
#tau = params$tau[1] # Optimize between 40-99
lm.searchwin = NULL
specT=30

# Extracts the image data as a matrix:
img <- matrix(dim(imagery)[2], dim(imagery)[1], data = imagery[,], byrow = FALSE)
Sobel <- matrix(dim(imagery)[2], dim(imagery)[1], data = im_sobel[,], byrow = FALSE)
img <- img[1:dim(imagery)[2], dim(imagery)[1]:1]
Sobel <- Sobel[1:dim(imagery)[2], dim(imagery)[1]:1]
img[is.na(img)] <- 0 # Sets any nas to 0s.
img[img < specT] <- 0 # Any values beneath the minimum height are set to 0s.

# Finds the local maxima:
coordSeeds <-  detect.maxima(img, res(imagery)[1], 
                             lm.searchwin = lm.searchwin, lut = lut, tau = tau)

# Segment crowns from local maxima
coordCrown <- segment.crowns(
  x = img,
  x.Sobel = Sobel,
  coordSeeds,
  THRESHSeed = params$THRESHSeed[1],
  THRESHCrown = params$THRESHCrown[1],
  SOBELstr = params$SOBELstr[1],
  scale = res(imagery)[1],
  lut = lut,
  tau = params$tau[1] # This one is now set and not related to tau in coordSeeds
)

crowns_sp<-crowns_to_spatial(coordCrown, imagery, pypath, convex.hull = FALSE)

file_name<-paste("/home/jovyan/lustre_scratch/bo/split_search_and_crown/uts/2021seed_", params[1,1],
                 '_crown_', params[1,2],
                 '_sobel_', params[1,3],
                 '_tau_', params[1,4],
                 '_searchwin_NULL',
                 '_specT_0', sep='')
writeOGR(crowns_sp,
         dsn = paste(file_name, '.shp', sep=''),
         layer = basename(file_name),
         drive = 'ESRI Shapefile')



out<-vector(mode="list", length=nrow(params))

### This is a one-off example - the for loop required
### to look at our initial parameter grid search is below.

manual<-readOGR("/home/jovyan/lustre_scratch/sepilok_data/sepilok_2014_manual_crowns/Manually_segmented_crowns/sepilok_603450_648000_manuals.shp")
crs(manual)<-crs(lid_chm)

ut<-readOGR(paste(file_name, '.shp', sep=''))

matched<-crown_overlap(auto_trees=ut, manual_trees=manual, buffer_by=0)
#plot(matched)
no_of_preds <- length(ut)
no_of_manuals <- length(manual)
matched<-matched[!is.na(matched$id_auto_tree),]
auto<-ut[matched$id_auto_tree,]
tp = length(matched)
fp = no_of_preds - tp
fn = no_of_manuals - length(auto)
  
precision = tp/(tp+fp)
recall = tp/(tp + fn)
fscore = (2*precision*recall)/(precision+recall)
min_fscore = 1 - fscore


# possibly for manual.id here, read in existing cost manual.ids 
# and add a cheeky one
out_tmp<-data.frame(manual.id=1126,
                    n.unmatch = sum(matched$tree_match),
                    mean.over = mean(matched$overseg),
                    sd.over = sd(matched$overseg),
                    mean.under = mean(matched$underseg),
                    sd.under = sd(matched$underseg),
                    mean.cost = mean(matched$cost),
                    med.cost = median(matched$cost),
                    sd.cost = sd(matched$cost),
                    mean.size_ratio = mean(matched$size_ratio),
                    med.size_ratio = median(matched$size_ratio),
                    sd.size_ratio = sd(matched$size_ratio),
                    min_f = min_fscore
)

out[[1]]<-out_tmp
# add the auto tree data to the manual trees:
matched<-matched[!is.na(matched$id_auto_tree),]
auto<-ut[matched$id_auto_tree,]
auto@data$shpbnd<-rowSums(auto@data[,c('hbnd_mn', 'hbnd_mx', 'soblbnd')])
auto@data$spcbnd<-rowSums(auto@data[,c('allmbnd', 'crwnbnd')])
auto@data<-auto@data[,c('trHghts_mn', 'trHghts_mx', 'R', 'shpbnd', 'spcbnd', 'sobl_mn')]
matched@data<-cbind(matched@data, auto@data)



cost<-do.call(rbind, out)
cost<-as.data.frame(cost)

old_cost<-read.csv("/home/jovyan/lustre_scratch/bo/split_search_and_crown/costs/low_tau/Cost_603450.csv")
new_cost<-rbind(old_cost[,c(2:14)], cost)

write.csv(new_cost, "/home/jovyan/lustre_scratch/bo/split_search_and_crown/costs/low_tau/Cost_603450.csv")

updated_cost<-read.csv("/home/jovyan/lustre_scratch/bo/split_search_and_crown/costs/low_tau/Cost_603450.csv")

params_cost<-cbind(params, cost)
#params_cost$cov.cost<-params_cost$sd.cost/params_cost$mean.cost

old_params_cost<-read.csv("/home/jovyan/lustre_scratch/bo/split_search_and_crown/costs/low_tau/Cost_params_603450.csv")
new_params_cost<-rbind(old_params_cost[,c(2:18)], params_cost)

write.csv(new_params_cost, "/home/jovyan/lustre_scratch/bo/split_search_and_crown/costs/low_tau/Cost_params_603450.csv")

In [ ]:
### testing occuring in this cell

### Need this cell the first time you run this bad boy

cost<-do.call(rbind, out)
cost<-as.data.frame(cost)
#cost

old_cost<-read.csv("/home/jovyan/lustre_scratch/bo/split_search_and_crown/costs/Cost_603450.csv")
#old_cost

new_cost<-rbind(old_cost[,c(2:14)], cost)
#new_cost

write.csv(new_cost, "/home/jovyan/lustre_scratch/bo/split_search_and_crown/costs/Cost_603450.csv")

updated_cost<-read.csv("/home/jovyan/lustre_scratch/bo/split_search_and_crown/costs/Cost_603450.csv")

params_cost<-cbind(params, cost)
#params_cost$cov.cost<-params_cost$sd.cost/params_cost$mean.cost

old_params_cost<-read.csv("/home/jovyan/lustre_scratch/bo/split_search_and_crown/costs/Cost_params_603450.csv")
#old_params_cost
#old_cost
#params_cost


new_params_cost<-rbind(old_params_cost[,c(2:5,7:19)], params_cost)
new_params_cost
write.csv(new_params_cost, "/home/jovyan/lustre_scratch/bo/split_search_and_crown/costs/Cost_params_603450.csv")